In [80]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np


from keras.utils import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout
from keras.metrics import Precision, Recall

import keras_tuner as kt
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [81]:
# get the data directories
dir = './archive'
train_dir = dir + '/train'
test_dir = dir + '/test'

In [82]:
# generate tf datasets

train = image_dataset_from_directory(
    train_dir,
    label_mode = 'binary',
    image_size = (32, 32)
)

test = image_dataset_from_directory(
    test_dir,
    label_mode = 'binary',
    image_size = (32, 32)
)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [83]:
# create a CNN Sequential model using Keras
model = Sequential([
    Rescaling(scale = 1./255),
    Conv2D(filters = 64, kernel_size = 3, activation = 'relu'),
    MaxPooling2D(),
    Conv2D(filters = 64, kernel_size = 3, activation = 'relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(units = 64),
    BatchNormalization(),
    Activation('relu'),
    Dense(units = 1, activation = 'sigmoid')
])

# compile the model with an optimizer, loss function, and metrics
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# train the model
model.fit(train, validation_data = test, epochs = 5)

Epoch 1/5
3125/3125 [==============================] - 548s 174ms/step - loss: 0.2741 - accuracy: 0.8850 - precision_6: 0.8831 - recall_6: 0.8874 - val_loss: 0.3052 - val_accuracy: 0.8720 - val_precision_6: 0.8134 - val_recall_6: 0.9654
Epoch 2/5
3125/3125 [==============================] - 130s 41ms/step - loss: 0.2082 - accuracy: 0.9161 - precision_6: 0.9153 - recall_6: 0.9170 - val_loss: 0.5983 - val_accuracy: 0.7849 - val_precision_6: 0.7010 - val_recall_6: 0.9935
Epoch 3/5
3125/3125 [==============================] - 127s 41ms/step - loss: 0.1845 - accuracy: 0.9270 - precision_6: 0.9261 - recall_6: 0.9280 - val_loss: 0.2979 - val_accuracy: 0.8838 - val_precision_6: 0.8213 - val_recall_6: 0.9811
Epoch 4/5
3125/3125 [==============================] - 131s 42ms/step - loss: 0.1679 - accuracy: 0.9333 - precision_6: 0.9321 - recall_6: 0.9346 - val_loss: 0.3802 - val_accuracy: 0.8595 - val_precision_6: 0.7860 - val_recall_6: 0.9879
Epoch 5/5
3125/3125 [==============================] - 

In [84]:
# evaluate the model on the test set
y_pred = model.predict(test)

625/625 [==============================] - 10s 15ms/step


In [85]:
from keras.metrics import Precision, Recall, BinaryAccuracy

#calculate precision, recall, and accuracy
pre = Precision()
rec = Recall()
acc = BinaryAccuracy()

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    rec.update_state(y, yhat)
    acc.update_state(y, yhat)
    
print(f'Recall: {rec.result().numpy()}, Precision: {pre.result().numpy()}, Accuracy: {acc.result().numpy()}')

1/1 [==============================] - 0s 40ms/step
Recall: 0.9735000133514404, Precision: 0.8351204991340637, Accuracy: 0.8906499743461609


In [86]:
import cv2

In [1]:
# function to predict whether image at the given path is real or fake
def predict_image(image_path):
    # load the image and resize it to 32x32
    img = cv2.imread(image_path)
    img = tf.image.resize(img, (32, 32))
    
    # predict the class
    y_prob = model.predict(np.expand_dims(img, 0))
    return 'REAL' if y_prob[0]>0.5 else 'FAKE'# function to predict whether image at the given path is real or fake

In [2]:
image_path = './archive/train/REAL/0000 (3).jpg'

In [165]:
import pickle

In [166]:
# save the model to a file
model_pkl_file = "model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)